#### Collapsing series of rank-one updates

We have a series of rank-one updates, but we would like to create an approximate rank-one update where some of the magnitude of the new data is squashed onto the dominant eigenvector. The new system's dominant eigenvalue will be an upper bound.

Rank-one update vectors are not actually additive. There could be consequences all across the spectrum after one rank-one update. We would need a tight upper bound for the dominant eigenvalue after one rank-one update, as well as an uncertainty (cosine difference tolerance) for its eigenvector. We could define an ellipse with the sqrt dominant eigenvalue as one axis and the sqrt trace minus dominant eigenvalue as the other axis. The eigenvalue and update vector component would be lower bounded and upper bounded to bound the new system.

In [74]:
import numpy as np
np.set_printoptions(linewidth = 150, precision = 4, suppress = True)

In [75]:
data = np.loadtxt('../NC-Data.csv', delimiter=',', dtype=str)
data = data[1:].astype(float)
data

array([[ 1.    , -0.0461,  0.2312, ...,  0.2704,  0.4664,  0.3672],
       [-0.0461,  1.    , -0.0671, ..., -0.0515, -0.0944, -0.0349],
       [ 0.2312, -0.0671,  1.    , ...,  0.147 ,  0.2608,  0.6313],
       ...,
       [ 0.2704, -0.0515,  0.147 , ...,  1.    ,  0.2066,  0.1538],
       [ 0.4664, -0.0944,  0.2608, ...,  0.2066,  1.    ,  0.3486],
       [ 0.3672, -0.0349,  0.6313, ...,  0.1538,  0.3486,  1.    ]])

In [76]:
cov = data[:3, :3]
cov

array([[ 1.    , -0.0461,  0.2312],
       [-0.0461,  1.    , -0.0671],
       [ 0.2312, -0.0671,  1.    ]])

In [77]:
d1, d2 = np.random.choice(np.arange(3, len(data)), 2, replace=0)
# Include a choice here which breaks.
d1, d2 = 85, 91
d1, d2

(85, 91)

In [78]:
eigvals, eigvecs = np.linalg.eigh(cov)
eigvals

array([0.7678, 0.9759, 1.2563])

In [79]:
proj_updates = eigvecs.T @ data[0:3, [d1, d2]] / eigvals[:, None]
proj_updates

array([[0.2212, 0.2206],
       [0.1236, 0.333 ],
       [0.2077, 0.0353]])

In [80]:
# Trivial upper bound on dominant eigval, from the trace.
eigvals[-1] + data[d1, d1] + data[d2, d2]

3.2562505988455444

In [81]:
# Actual dominant eigenvalue.
np.linalg.eigvalsh(data[[0, 1, 2, d1, d2], :][:, [0, 1, 2, d1, d2]])[-1]

1.7439990337198588

In [82]:
def resize(arr, size):
    # Pad tuple: zeroes inserted before, and zeroes inserted after.
    return np.pad(arr, [(0, new_len - arr.shape[i]) for i, new_len in enumerate(size)])

In [90]:
# L1 norm (sum of absolute values of updates).
proj_update_rank_one_bounded = np.linalg.norm(proj_updates, ord=1, axis=1, keepdims=1)
proj_update_rank_one_bounded = np.r_[
    proj_update_rank_one_bounded,
    [[np.sqrt(data[d1, d1] + data[d2, d2]) - np.linalg.norm(proj_update_rank_one_bounded)]],
    [[0]],
]
resize(cov, [5, 5])
proj_update_rank_one_bounded

array([[0.4418],
       [0.4566],
       [0.243 ],
       [0.734 ],
       [0.    ]])

In [91]:
eigvals_spectrum = np.linalg.eigvalsh(
    resize(cov, [5, 5])
    + proj_update_rank_one_bounded @ proj_update_rank_one_bounded.T)
np.testing.assert_array_less(
    np.linalg.eigvalsh(data[[0, 1, 2, d1, d2], :][:, [0, 1, 2, d1, d2]])[-1],
    eigvals_spectrum[-1])
eigvals_spectrum[-1]

AssertionError: 
Arrays are not less-ordered

Mismatched elements: 1 / 1 (100%)
Max absolute difference: 0.0105
Max relative difference: 0.006
 x: array(1.743999)
 y: array(1.733546)